In [1]:
from datetime import datetime, date, timedelta
import pandas as pd
import geopandas as gpd
from random import randint
print('All imported')

All imported


In [8]:
papka = str(input())
nomer = int(input())

 C:/Users/Sirius/Desktop/Тест временные
 62138


In [3]:
def create_table_cup(input_file= papka + '/warnings.txt'):
    file = open(input_file)
    tcu = []
    opd = []
    no_info_strings = 0
    step = 0
    for i in file:
        if i[0] in '0123456789':
            parts = i.split()
            date_start_list = parts[0].split("T")
            date_end_list = parts[1].split("T")
            date_start = datetime(int(date_start_list[0][:4]), int(date_start_list[0][5:7]), int(date_start_list[0][8:10]), int(date_start_list[1][:2]), int(date_start_list[1][3:5]), int(date_start_list[1][6:8]))
            date_end = datetime(int(date_end_list[0][:4]), int(date_end_list[0][5:7]), int(date_end_list[0][8:10]), int(date_end_list[1][:2]), int(date_end_list[1][3:5]), int(date_end_list[1][6:8]))
            if no_info_strings < 5:
                tcu.append([date_start, date_end])
            else:
                opd.append([date_start, date_end])
        else:
            no_info_strings += 1
    return tcu, opd

In [4]:
def is_time_between(begin_time, end_time, check_time):
    if begin_time < end_time:
        return check_time >= begin_time and check_time <= end_time
    else:
        return check_time >= begin_time or check_time <= end_time

In [13]:
def cup_intersections(input_subsatellite_points= papka + '/trassirovka_{}.shp'.format(nomer), output_file=papka + '/CUP/'):
    subs_points = gpd.read_file(input_subsatellite_points)
    cup_tcu, cup_opd = create_table_cup()
    step = 0
    for time_interval in cup_tcu:
        cup_points = subs_points
        for id, point in subs_points.iterrows():
            point_date_time = datetime.strptime(point["TIME"], '%Y-%m-%d %H:%M:%S')
            if not is_time_between(time_interval[0], time_interval[1], point_date_time):
                cup_points = cup_points.drop(axis=0, index=id, inplace=False)
        cup_points.to_file(str(output_file) + 'tcu_points' + str(step) + '.shp')
        step += 1
    step = 0
    for time_interval in cup_opd:
        cup_points = subs_points
        for id, point in subs_points.iterrows():
            point_date_time = datetime.strptime(point["TIME"], '%Y-%m-%d %H:%M:%S')
            if not is_time_between(time_interval[0], time_interval[1], point_date_time):
                cup_points = cup_points.drop(axis=0, index=id, inplace=False)
        cup_points.to_file(str(output_file) + 'opd_points' + str(step) + '.shp')
        step += 1

In [15]:
cup_intersections()

In [17]:
def success_shooting(time_delta=81, shooting_programm=papka + '/shooting_time_all.csv', correct_shooting_program=papka + '/corrected_shooting_points_1.xlsx'):
    tcu, opd = create_table_cup()
    restricts = tcu + opd
    shooting_input = pd.read_csv(shooting_programm)
    shooting_table = pd.DataFrame(shooting_input)
    dates_of_shooting = []
    for id, row in shooting_table.iterrows():
        dates_of_shooting.append(datetime.strptime(row["TIME"], '%Y-%m-%d %H:%M:%S'))
    dates_of_shooting.sort()
    is_valid_coloumn = []
    time_delta_valid = []
    reason_of_false = []
    last_date_of_shooting = None
    # print('START\t\t\tEND\t\t\tCHECK\t\t\tSTATE')
    for row in dates_of_shooting:
        all = 0
        reason = None
        for i in restricts:
            if not is_time_between(i[0], i[1], row):
                all += 1
            else:
                reason = "TCU" if i in tcu else "OPD"
            # print(str(i[0]) + '\t' + str(i[1]) + '\t' + str(row) + '\t' + str(True if not is_time_between(i[0], i[1], row) else False))
        
        if all == len(restricts):
            is_valid_coloumn.append(True)
            reason_of_false.append(None)
        else:
            is_valid_coloumn.append(False)
            reason_of_false.append(reason)
        try:
            if int((row - last_date_of_shooting).total_seconds()) <= time_delta:
                # time_delta_valid[-1] = False
                time_delta_valid.append(False)
            else:
                time_delta_valid.append(True)
                last_date_of_shooting = row
        except:
            time_delta_valid.append(True)
            last_date_of_shooting = row
    is_valid_series = pd.Series(is_valid_coloumn, name="IS_VALID")
    time_delta_valid_series = pd.Series(time_delta_valid, name='TM_DLT_VLD')
    shooting_table["TIME"] = dates_of_shooting
    shooting_table["IS_VALID"] = is_valid_series
    shooting_table["TM_DLT_VLD"] = time_delta_valid_series
    shooting_table["REASON"] = reason_of_false
    shooting_table.to_excel(correct_shooting_program, index=False)

In [19]:
success_shooting()